In [3]:
import os

os.chdir('/home/beihang/xihu/HZTourism/FlowPred-dev')

In [2]:
import pymysql
import csv

DB_CONFIG = {
    'host': "10.62.193.1",
    'port': 3306,
    'user': "xihuspot",
    'password': "xihu#123",
    'database': "flowpredicationsa", # <-- 需要导出数据的数据库名
    'charset': 'utf8mb4', # 建议使用 utf8mb4 以支持各种字符
    'cursorclass': pymysql.cursors.DictCursor # 使用 DictCursor 可以让每行数据都像字典一样，方便按列名访问
}

TABLE_TO_EXPORT = 'hz_weather_7_forecast' # <-- 您想导出的表名
OUTPUT_CSV_FILE = 'data/wheather_enhence/raw/all/wheather_230519_250705.csv' # <-- 输出的 CSV 文件名

In [3]:
try:
    # 连接数据库
    connection = pymysql.connect(**DB_CONFIG)
    print("数据库连接成功！")

    with connection.cursor() as cursor:
        # 查询表中的所有数据
        print(f"正在从表 '{TABLE_TO_EXPORT}' 中查询数据...")
        cursor.execute(f"SELECT * FROM {TABLE_TO_EXPORT}")
        
        # 获取第一行数据来确定表头（列名）
        first_row = cursor.fetchone()
        
        if first_row:
            headers = list(first_row.keys())
            print(f"获取到表头: {headers}")

            # 打开 CSV 文件准备写入
            with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8-sig') as csvfile:
                # 注意：
                # newline='' 是为了防止写入时出现多余的空行
                # encoding='utf-8-sig' 可以让 Excel 正确识别 UTF-8 编码，避免中文乱码
                
                writer = csv.writer(csvfile)

                # 写入表头
                writer.writerow(headers)
                print("CSV 文件表头已写入。")
                
                # 写入第一行数据
                writer.writerow(first_row.values())
                
                # 循环写入剩余的所有数据
                # 使用 cursor.fetchall() 一次性获取所有剩余行
                remaining_rows = cursor.fetchall()
                for row in remaining_rows:
                    writer.writerow(row.values())
            
            print(f"成功！所有数据已导出到文件 '{OUTPUT_CSV_FILE}'。")
        else:
            print(f"表 '{TABLE_TO_EXPORT}' 中没有数据。")

except pymysql.Error as err:
    print(f"数据库操作失败: {err}")
except Exception as e:
    print(f"发生未知错误: {e}")
finally:
    # 关闭数据库连接
    if 'connection' in locals() and connection.open:
        connection.close()
        print("数据库连接已关闭。")

数据库连接成功！
正在从表 'hz_weather_7_forecast' 中查询数据...
获取到表头: ['id', 'StartTime', 'ProductTime', 'data', 'Zname', 'Region', 'station', 'Lon', 'lat', 'Altitude', 'City', 'Province', 'updatetime']
CSV 文件表头已写入。
成功！所有数据已导出到文件 'data/wheather_enhence/raw/all/wheather_230519_250705.csv'。
数据库连接已关闭。


In [4]:
import pandas as pd

df = pd.read_csv("data/wheather_enhence/raw/all/wheather_230519_250705.csv")

In [5]:
df["data"][0]

'[{"Rain12":0.0,"WindLevel12Code":0.0,"Weather12":"阴","Weather12Code":2.0,"WindDirect12":"","WindLevel12":"不超过3级","ForecastSpan":12,"WindDirect12Code":0.0},{"Rain12":0.0,"TempHigh":28.4,"WindLevel12Code":1.0,"Weather12":"阴","Weather12Code":2.0,"TempLow":17.4,"WindDirect12":"东风","WindLevel12":"3-4级","ForecastSpan":24,"WindDirect12Code":2.0},{"Rain12":0.0,"WindLevel12Code":0.0,"Weather12":"阴","Weather12Code":2.0,"WindDirect12":"","WindLevel12":"不超过3级","ForecastSpan":36,"WindDirect12Code":0.0},{"Rain12":8.2,"TempHigh":28.4,"WindLevel12Code":1.0,"Weather12":"小雨到中雨","Weather12Code":21.0,"TempLow":19.9,"WindDirect12":"东南风","WindLevel12":"3-4级","ForecastSpan":48,"WindDirect12Code":3.0},{"Rain12":21.6,"WindLevel12Code":3.0,"Weather12":"中雨","Weather12Code":8.0,"WindDirect12":"北风","WindLevel12":"5-6级","ForecastSpan":60,"WindDirect12Code":8.0},{"Rain12":7.1,"TempHigh":23.6,"WindLevel12Code":2.0,"Weather12":"小雨","Weather12Code":7.0,"TempLow":17.2,"WindDirect12":"北风","WindLevel12":"4-5级","ForecastS

In [11]:
import json
data_list = json.loads(df["data"][0])
data_list1 = json.loads(df["data"][1])
data_list2 = json.loads(df["data"][2])

In [12]:
len(data_list2)

14

In [8]:
for item in data_list1:
    print(item)

{'Rain12': 7.6, 'WindLevel12Code': 3.0, 'Weather12': '小雨', 'Weather12Code': 7.0, 'WindDirect12': '北风', 'WindLevel12': '5-6级', 'ForecastSpan': 12, 'WindDirect12Code': 8.0}
{'Rain12': 0.0, 'TempHigh': 20.6, 'WindLevel12Code': 0.0, 'Weather12': '多云', 'Weather12Code': 1.0, 'TempLow': 14.2, 'WindDirect12': '', 'WindLevel12': '不超过3级', 'ForecastSpan': 24, 'WindDirect12Code': 0.0}
{'Rain12': 0.0, 'WindLevel12Code': 0.0, 'Weather12': '多云', 'Weather12Code': 1.0, 'WindDirect12': '', 'WindLevel12': '不超过3级', 'ForecastSpan': 36, 'WindDirect12Code': 0.0}
{'Rain12': 0.0, 'TempHigh': 24.1, 'WindLevel12Code': 0.0, 'Weather12': '多云', 'Weather12Code': 1.0, 'TempLow': 15.0, 'WindDirect12': '', 'WindLevel12': '不超过3级', 'ForecastSpan': 48, 'WindDirect12Code': 0.0}
{'Rain12': 0.0, 'WindLevel12Code': 0.0, 'Weather12': '多云', 'Weather12Code': 1.0, 'WindDirect12': '', 'WindLevel12': '不超过3级', 'ForecastSpan': 60, 'WindDirect12Code': 0.0}
{'Rain12': 0.0, 'TempHigh': 23.9, 'WindLevel12Code': 1.0, 'Weather12': '多云', 'W

## 对天气预报数据进行处理

In [51]:
import pandas as pd

df = pd.read_csv("data/wheather_enhence/ogn/all/wheather_230519_250705.csv")
df["StartTime"] = pd.to_datetime(df["StartTime"])
# 筛选24年以后的数据
df_24_25 = df[df["StartTime"] >= pd.to_datetime("2023-12-31")]
df_24_25.drop_duplicates(subset=["StartTime"], inplace=True)
# 只保留StartTime、data
df_24_25 = df_24_25[["StartTime","data"]]

/tmp/ipykernel_1589488/6939056.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_24_25.drop_duplicates(subset=["StartTime"], inplace=True)


In [52]:
import numpy as np
import json

pred_time_list =  []
start_time_list = []
end_time_list = []

weather_list=[]
# 降雨量
rain_list = []
# 风力
wind_level_list = []
# 风向
wind_direc_list = []
# 最高温
temp_max_list = []
# 最低温
temp_min_list = []

for row in df_24_25.itertuples():
    start_time = row.StartTime
    pred_time = start_time
    weather_data = json.loads(row.data)
    for i,item in enumerate(weather_data):
        # kpi_time=kpi_time+12小时*i
        end_time = start_time + pd.Timedelta(hours=12)
        pred_time_list.append(pred_time)
        start_time_list.append(start_time)
        end_time_list.append(end_time)
        rain_list.append(item["Rain12"])
        weather_list.append(item["Weather12Code"])
        wind_level_list.append(item["WindLevel12Code"])
        wind_direc_list.append(item["WindDirect12Code"])
        temp_max_list.append(item.get("TempHigh",np.NaN))
        temp_min_list.append(item.get("TempLow",np.NaN))
        start_time = start_time + pd.Timedelta(hours=12)
    

In [53]:
# 构建一个dataframe
df_24_25_res = pd.DataFrame({
    "pred_time":pred_time_list,
    "start_time":start_time_list,
    "end_time":end_time_list,
    "rain":rain_list,
    "weather":weather_list,
    "wind_level":wind_level_list,
    "wind_direc":wind_direc_list,
    "temp_max":temp_max_list,
    "temp_min":temp_min_list
})

df_24_25_res.fillna(method="bfill",inplace=True)

In [56]:
df_24_25_res.to_csv("data/wheather_enhence/ogn/all/weather_24_25_preprocessed.csv",index=False)

## 将天气数据整合进客流量数据

In [71]:
df_24 = pd.read_csv("data/wheather_enhence/ogn/24/df_2024.csv")
df_25 = pd.read_csv("data/wheather_enhence/ogn/25/df_2025.csv")
df_weather = pd.read_csv("data/wheather_enhence/ogn/all/weather_24_25_preprocessed.csv")

In [72]:
def get_future_weather_features(
    df_weather: pd.DataFrame, 
    prediction_start_time: str,
    time_step: str, 
    num_steps_to_predict: int
) -> pd.DataFrame:
    """
    为一个客流预测任务，批量获取未来多个时间片的天气特征。

    :param df_weather: 原始的天气预测DataFrame。
    :param prediction_start_time: 客流预测的起始时间点。
    :param time_step: 每个预测时间片的大小（分钟或者秒）m/s。
    :param num_steps_to_predict: 需要预测的未来时间片的数量。
    :return: 一个包含未来所有时间点及其对应天气特征的DataFrame。
    """
    
    start_time = pd.to_datetime(prediction_start_time)
    df_weather["pred_time"] = pd.to_datetime(df_weather["pred_time"])
    df_weather["start_time"] = pd.to_datetime(df_weather["start_time"])
    df_weather["end_time"] = pd.to_datetime(df_weather["end_time"])

    # 步骤1: 回溯筛选 - 只使用在客流预测开始前已发布的天气预报
    available_forecasts = df_weather[df_weather['pred_time'] < start_time].copy()
    
    if available_forecasts.empty:
        print(f"警告: 在 {start_time} 之前没有任何天气预报数据。无法生成特征。")
        return pd.DataFrame()

    if 'm' in time_step:
        time_step_res = int(time_step.replace('m', ''))
        unit='m'
    elif 's' in time_step:
        time_step_res = int(time_step.replace('s', ''))
        unit='s'
    else:
        raise ValueError("时间步长必须以 'm' 或 's' 为单位")

    # 准备一个列表来存储每一片的查找结果
    results_list = []
    
    # 步骤2: 逐片查找 - 遍历未来需要预测的每一个时间片
    for i in range(num_steps_to_predict):
        # 计算当前要查找的目标时间点
        target_dt = start_time + pd.to_timedelta(i * time_step_res, unit=unit)
        
        # --- 内层逻辑：为单个时间点查找最佳预测 ---
        # 筛选出所有时间段覆盖了目标时间点的候选预测
        candidates = available_forecasts[
            (available_forecasts['start_time'] <= target_dt) & 
            (available_forecasts['end_time'] > target_dt)
        ].copy()

        best_forecast_row = None
        if not candidates.empty:
            # 从候选中选择 pred_time 最晚（最新）的那一条
            best_forecast_row = candidates.sort_values(by='pred_time', ascending=False).iloc[0]
        
        # --- 结果处理 ---
        if best_forecast_row is not None:
            # 提取天气指标
            weather_features = best_forecast_row.drop(['pred_time', 'start_time', 'end_time']).to_dict()
            weather_features['kpi_time'] = target_dt # 添加目标时间戳
            results_list.append(weather_features)
        else:
            # 如果没找到，也要记录下来，用NaN填充
            print(f"警告: 未能为 {target_dt} 找到任何可用的天气预报。")
            # 获取天气特征列的列名
            weather_cols = df_weather.columns.drop(['pred_time', 'start_time', 'end_time'])
            missing_features = {col: np.nan for col in weather_cols}
            missing_features['kpi_time'] = target_dt
            results_list.append(missing_features)

    # 将所有结果合并成一个最终的DataFrame
    final_df = pd.DataFrame(results_list)
    
    # 重新排序列，让kpi_time在第一列
    if 'kpi_time' in final_df.columns:
        cols = ['kpi_time'] + [col for col in final_df.columns if col != 'kpi_time']
        final_df = final_df[cols]
        
    return final_df


In [73]:
df_weather

,pred_time,start_time,end_time,rain,weather,wind_level,wind_direc,temp_max,temp_min
0,2023-12-31 08:00:00,2023-12-31 08:00:00,2023-12-31 20:00:00,0.0,0.0,2.0,1.0,13.0,1.6
1,2023-12-31 08:00:00,2023-12-31 20:00:00,2024-01-01 08:00:00,0.0,0.0,0.0,0.0,13.0,1.6
2,2023-12-31 08:00:00,2024-01-01 08:00:00,2024-01-01 20:00:00,0.0,1.0,0.0,0.0,9.2,0.5
3,2023-12-31 08:00:00,2024-01-01 20:00:00,2024-01-02 08:00:00,0.0,1.0,0.0,0.0,9.2,0.5
4,2023-12-31 08:00:00,2024-01-02 08:00:00,2024-01-02 20:00:00,0.0,2.0,1.0,8.0,12.4,2.2
...,...,...,...,...,...,...,...,...,...
9263,2025-07-08 06:45:00,2025-07-12 18:45:00,2025-07-13 06:45:00,0.0,1.0,0.0,5.0,32.1,25.0
9264,2025-07-08 06:45:00,2025-07-13 06:45:00,2025-07-13 18:45:00,0.0,8.0,0.0,1.0,32.2,25.4
9265,2025-07-08 06:45:00,2025-07-13 18:45:00,2025-07-14 06:45:00,0.0,1.0,0.0,5.0,32.2,25.4
9266,2025-07-08 06:45:00,2025-07-14 06:45:00,2025-07-14 18:45:00,0.0,7.0,0.0,3.0,33.4,26.0


In [85]:
prediction_start_time=pd.to_datetime("2025-05-05 09:50:00")
time_step = "5m"
num_steps_to_predict=72

df_weather_future = get_future_weather_features(
    df_weather=df_weather,
    prediction_start_time=prediction_start_time,
    time_step=time_step,
    num_steps_to_predict=num_steps_to_predict
)


In [86]:
df_weather_future

,kpi_time,rain,weather,wind_level,wind_direc,temp_max,temp_min
0,2025-05-05 09:50:00,0.0,7.0,0.0,8.0,25.9,18.6
1,2025-05-05 09:55:00,0.0,7.0,0.0,8.0,25.9,18.6
2,2025-05-05 10:00:00,0.0,7.0,0.0,8.0,25.9,18.6
3,2025-05-05 10:05:00,0.0,7.0,0.0,8.0,25.9,18.6
4,2025-05-05 10:10:00,0.0,7.0,0.0,8.0,25.9,18.6
...,...,...,...,...,...,...,...
67,2025-05-05 15:25:00,0.0,7.0,0.0,8.0,25.9,18.6
68,2025-05-05 15:30:00,0.0,7.0,0.0,8.0,25.9,18.6
69,2025-05-05 15:35:00,0.0,7.0,0.0,8.0,25.9,18.6
70,2025-05-05 15:40:00,0.0,7.0,0.0,8.0,25.9,18.6
